In [5]:
import pandas as pd

In [10]:
cruces = pd.read_csv('cruces.csv', sep=';', encoding='latin1')
display(cruces)

,Codigo de vía tratado,Literal completo del vial tratado,Clase de la via tratado,Particula de la via tratado,Nombre de la via tratado,Codigo de via que cruza o enlaza,Literal completo del vial que cruza,Clase de la via que cruza,Particula de la via que cruza,Nombre de la via que cruza,Coordenada X (Guia Urbana) cm (cruce),Coordenada Y (Guia Urbana) cm (cruce),Longitud en S R WGS84 (cruce),Latitud en S R WGS84 (cruce)
0,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,2792,CALLE DE CASTIELLO DE JACA ...,CALLE,DE,CASTIELLO DE JACA,44260746,448479753,3º40'43.38'' W,40º30'36.13'' N
1,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,1837,CALLE DE FEDERICO MOMPOU ...,CALLE,DE,FEDERICO MOMPOU,44260732,448479748,3º40'43.43'' W,40º30'36.13'' N
2,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,87100,CALLE DE MARÍA TUBAU ...,CALLE,DE,MARIA TUBAU,44290433,448494484,3º40'30.85'' W,40º30'40.98'' N
3,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,2793,PASEO DE TIERRA DE MELIDE ...,PASEO,DE,TIERRA DE MELIDE,44320997,448510385,3º40'17.9'' W,40º30'46.21'' N
4,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,2794,CALLE DE PORTOMARÍN ...,CALLE,DE,PORTOMARIN,44322285,448530840,3º40'17.42'' W,40º30'52.83'' N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36160,31003706,CALLE DE TREN OBRERO ...,CALLE,DE,TREN OBRERO,31003703,CALLE DE JOSÉ MARÍA LOZANO SAINZ ...,CALLE,DE,JOSE MARIA LOZANO SAINZ,44083665,446665463,3º41'52.54'' W,40º20'47.3'' N
36161,31004081,GLORIETA DE CIRILO MARTÍN MARTÍN ...,GLORIETA,DE,CIRILO MARTIN MARTIN,369600,CARRETERA DE HÚMERA ...,CARRETERA,DE,HUMERA,43390886,447883842,3º46'50.75'' W,40º27'20.55'' N
36162,31005685,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ ...,CALLE,DE,ISIDORO ALVAREZ ALVAREZ,29900,CALLE DE ALICANTE ...,CALLE,DE,ALICANTE,44145890,447199432,3º41'27.95'' W,40º23'40.61'' N
36163,31005685,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ ...,CALLE,DE,ISIDORO ALVAREZ ALVAREZ,2729,CALLE DEL GRANITO ...,CALLE,DEL,GRANITO,44150370,447189024,3º41'26.0'' W,40º23'37.24'' N


In [7]:
direcciones = pd.read_csv('direcciones.csv', sep=';', encoding='latin1')
display(direcciones)

C:\Users\Alex MGG\AppData\Local\Temp\ipykernel_30384\497320727.py:1: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  direcciones = pd.read_csv('direcciones.csv', sep=';', encoding='latin1')


,Codigo de numero,Codigo de via,Clase de la via,Partícula de la vía,Nombre de la vía,Literal de numeracion,Codigo de distrito,Nombre del distrito,Codigo de barrio,Nombre del barrio,...,Coordenada X (Guia Urbana) cm,Coordenada Y (Guia Urbana) cm,Longitud en S R ETRS89 WGS84,Latitud en S R ETRS89 WGS84,Tipo de la via a la que pertenece el numero,Situacion de la via respecto al terreno,Tipo de denominacion de la via,Parcela catastral del numero,Tipologia del numero,Zona de valor del numero
0,31031089,31001337,AUTOVIA,,A-1,KM.001000EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,...,44305633,448250340,3º40'23.6'' W,40º29'21.82'' N,Topónimo,Nivel,Admon,,Parcela,R21S
1,31031088,31001337,AUTOVIA,,A-1,KM.001000SA,16,HORTALEZA,6,VALDEFUENTES,...,44312246,448249077,3º40'20.75'' W,40º29'21.45'' N,Topónimo,Nivel,Admon,,Parcela,R21N
2,31031091,31001337,AUTOVIA,,A-1,KM.001100EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,...,44367522,448330933,3º39'57.57'' W,40º29'48.11'' N,Topónimo,Nivel,Admon,,Parcela,R21S
3,31031090,31001337,AUTOVIA,,A-1,KM.001100SA,16,HORTALEZA,6,VALDEFUENTES,...,44373755,448325151,3º39'54.87'' W,40º29'46.28'' N,Topónimo,Nivel,Admon,,Parcela,R21R
4,31031093,31001337,AUTOVIA,,A-1,KM.001200EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,...,44425207,448408185,3º39'33.3'' W,40º30'13.32'' N,Topónimo,Nivel,Admon,,Parcela,R21O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210358,20045135,725,PASEO,DEL,ZURRON,NUM000049,21,BARAJAS,4,TIMON,...,45051964,448025888,3º35'5.92'' W,40º28'10.77'' N,Vía,Nivel,Pleno,0403807VK5800C,Frente fachada,R23T
210359,20045138,725,PASEO,DEL,ZURRON,NUM000051,21,BARAJAS,4,TIMON,...,45058080,448025960,3º35'3.33'' W,40º28'10.78'' N,Vía,Nivel,Pleno,0602301VK5800D,Frente fachada,R23T
210360,20045141,725,PASEO,DEL,ZURRON,NUM000053,21,BARAJAS,4,TIMON,...,45061025,448027575,3º35'2.11'' W,40º28'11.34'' N,Vía,Nivel,Pleno,0602301VK5800D,Portal,R23T
210361,20045144,725,PASEO,DEL,ZURRON,NUM000055,21,BARAJAS,4,TIMON,...,45063500,448027080,3º35'1.04'' W,40º28'11.18'' N,Vía,Nivel,Pleno,0602302VK5800D,Frente fachada,R23T


Unique de las coord
for coord:
    grafo.add_vertice(df[df['coord'] == coord])